In [ ]:
from google.colab import drive
import os## judt mounting drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q ultralytics boxmot filterpy ## using boxmot as the major repo for this task

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
import cv2 ## for dwawing out bbox and extracting frames
import numpy as np
from ultralytics import YOLO
from tqdm import tqdm  # For a nice progress bar

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from types import SimpleNamespace
from boxmot import StrongSort

print("Loading models and initializing tracker...")

Loading models and initializing tracker...


In [ ]:
PROJECT_PATH = '/content/drive/My Drive/detect'
re_id_model= 'osnet_ibn_x1_0_msmt17.pt' ## just for start lets see , i like to use vit or this but lets start slow
yolo_model = 'best.pt'

In [ ]:
YOLO_MODEL_PATH = os.path.join(PROJECT_PATH, yolo_model)
REID_MODEL_PATH = os.path.join(PROJECT_PATH, re_id_model)
INPUT_VIDEO_PATH = os.path.join(PROJECT_PATH, '15sec_input_720p.mp4')
OUTPUT_VIDEO_PATH = os.path.join(PROJECT_PATH, 'output_9_test.mp4')  ## output as of now

In [ ]:
!ls '/content/drive/My Drive/detect'

15sec_input_720p.mp4
best.pt
msmt_bot_R50.pt
osnet_ibn_x1_0_msmt17.pt
osnet_x1_0_msmt17_256x128_amsgrad_ep150_stp60_lr0.0015_b64_fb10_softmax_labelsmooth_flip.pth
osnet_x1_0_msmt17_combineall.pt
osnet_x1_0_msmt17.pt
output_1.mp4
output_2.mp4
output_3.mp4
output_4.mp4
output_5.mp4
output_6.mp4
output_7.mp4
output_8_HOPE.mp4
output.mp4


In [ ]:
from pathlib import Path
import torch


In [ ]:

def magic_happens(video_path, yolo_path, output_path, tracker_config):
    """
    Main function to process the video with added print statements for debugging.
    """
    ##Model and Tracker Initialization
    print("Loading models and initializing tracker...")
    try:
        yolo_model = YOLO(yolo_path)
        print("YOLO model loaded successfully.")
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return

    try:
        tracker = StrongSort(**tracker_config)
        print("StrongSORT tracker initialized successfully.")
    except Exception as e:
        print(f"Error initializing tracker: {e}")
        raise

    print("Opening video source...")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    print("Opening video source...")
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    print("Starting video processing with debug output...")
    frame_num = 0
    with tqdm(total=total_frames, desc="Processing Frames") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_num += 1
            print(f"\n--- Processing Frame {frame_num} ---")


            detections = yolo_model(frame, verbose=False)[0]

            print(f"DEBUG: YOLO found {len(detections.boxes)} total objects in this frame.")
            if len(detections.boxes) > 0:
                detected_classes = [int(box.cls) for box in detections.boxes]
                print(f"DEBUG: Detected class IDs are: {list(set(detected_classes))}")


            ##FORMATTING
            player_detections = []
            for box in detections.boxes:

                if int(box.cls) == 2: ## finding this class 2 was a a bit huddle
                    player_detections.append(box.data.tolist()[0])

            print(f"DEBUG: After filtering for player class (ID 2), we have {len(player_detections)} detections.")

            tracks = []
            if len(player_detections) > 0:
                detections_np = np.array(player_detections)

                tracks = tracker.update(detections_np, frame)
                print(f"DEBUG: StrongSORT tracker returned {len(tracks)} tracks for this frame.")
            else:
                # If no players were detected, the tracker is not even called.
                tracks = tracker.update(np.empty((0, 6)), frame)
                print(f"DEBUG: No players detected, updating tracker with empty data.")


            ##VISUALIZATION
            if len(tracks) > 0:
                for track in tracks:
                    x1, y1, x2, y2, track_id = map(int, track[:5])
                    print(f"DEBUG: Drawing track ID {track_id} at bbox [{x1}, {y1}, {x2}, {y2}]")
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            out.write(frame)
            pbar.update(1)

    # Release resources
    cap.release()
    out.release()
    print(f"\nProcessing complete. Output video saved to: {output_path}")

if __name__ == "__main__":
    os.makedirs(os.path.dirname(OUTPUT_VIDEO_PATH), exist_ok=True)

    ##This dictionary now holds all the settings.
    tracker_config_for_init = {
        'reid_weights': Path(REID_MODEL_PATH),
        'device': torch.device('cuda:0'),
        'half': True,
        'max_age': 150,
        'n_init': 3,
        'max_cos_dist': 0.30,# Appearance distance threshold
        'min_conf': 0.5,## detection threhold
        'max_iou_dist': 0.75, ## dist threhold
        'mc_lambda': 0.93
    }

    # Check if files exist before running
    if os.path.exists(INPUT_VIDEO_PATH) and os.path.exists(YOLO_MODEL_PATH) and os.path.exists(REID_MODEL_PATH):
        magic_happens(INPUT_VIDEO_PATH, YOLO_MODEL_PATH, OUTPUT_VIDEO_PATH, tracker_config_for_init)
    else:
        print("Error: One or more required files (video, YOLO model, Re-ID model) were not found. Please check your paths.")

Loading models and initializing tracker...


2025-07-02 21:08:50.810 | INFO     | boxmot.utils.torch_utils:select_device:78 - Yolo Tracking v13.0.14 🚀 Python-3.11.13 torch-2.6.0+cu124
CUDA:0 (Tesla T4, 15095MiB)
2025-07-02 21:08:50.811 | ERROR    | boxmot.appearance.backends.base_backend:download_model:152 - Found existing ReID weights at /content/drive/My Drive/detect/osnet_ibn_x1_0_msmt17.pt; skipping download.


YOLO model loaded successfully.


2025-07-02 21:08:52.509 | SUCCESS  | boxmot.appearance.reid.registry:load_pretrained_weights:64 - Loaded pretrained weights from /content/drive/My Drive/detect/osnet_ibn_x1_0_msmt17.pt


StrongSORT tracker initialized successfully.
Opening video source...
Opening video source...
Starting video processing with debug output...


...

DEBUG: StrongSORT tracker returned 12 tracks for this frame.
DEBUG: Drawing track ID 8 at bbox [186, 259, 211, 355]
DEBUG: Drawing track ID 37 at bbox [326, 348, 381, 460]
DEBUG: Drawing track ID 67 at bbox [22, 235, 79, 317]
DEBUG: Drawing track ID 187 at bbox [677, 284, 718, 395]
DEBUG: Drawing track ID 198 at bbox [502, 183, 516, 207]
DEBUG: Drawing track ID 208 at bbox [626, 257, 665, 360]
DEBUG: Drawing track ID 231 at bbox [125, 252, 167, 316]
DEBUG: Drawing track ID 248 at bbox [186, 301, 209, 357]
DEBUG: Drawing track ID 250 at bbox [258, 301, 292, 366]
DEBUG: Drawing track ID 272 at bbox [131, 415, 187, 541]
DEBUG: Drawing track ID 273 at bbox [120, 290, 161, 395]
DEBUG: Drawing track ID 289 at bbox [118, 292, 155, 364]

--- Processing Frame 371 ---
DEBUG: YOLO found 12 total objects in this frame.
DEBUG: Detected class IDs are: [2]
DEBUG: After filtering for player class (ID 2), we have 12 detections.


Processing Frames:  99%|█████████▉| 371/375 [02:54<00:02,  1.55it/s]2025-07-02 21:11:48.304 | WARNING  | boxmot.motion.cmc.ecc:apply:99 - Affine matrix could not be generated: OpenCV(4.11.0) /io/opencv/modules/video/src/ecc.cpp:589: error: (-7:Iterations do not converge) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function 'findTransformECC'
. Returning identity
Processing Frames:  99%|█████████▉| 372/375 [02:54<00:01,  2.05it/s]

DEBUG: StrongSORT tracker returned 10 tracks for this frame.
DEBUG: Drawing track ID 8 at bbox [185, 255, 211, 355]
DEBUG: Drawing track ID 37 at bbox [339, 344, 390, 451]
DEBUG: Drawing track ID 187 at bbox [673, 279, 715, 391]
DEBUG: Drawing track ID 198 at bbox [502, 181, 516, 205]
DEBUG: Drawing track ID 208 at bbox [629, 253, 669, 357]
DEBUG: Drawing track ID 231 at bbox [115, 246, 170, 332]
DEBUG: Drawing track ID 250 at bbox [257, 299, 291, 364]
DEBUG: Drawing track ID 272 at bbox [130, 410, 190, 538]
DEBUG: Drawing track ID 273 at bbox [120, 287, 162, 391]
DEBUG: Drawing track ID 291 at bbox [32, 223, 104, 314]

--- Processing Frame 372 ---
DEBUG: YOLO found 0 total objects in this frame.
DEBUG: After filtering for player class (ID 2), we have 0 detections.
DEBUG: No players detected, updating tracker with empty data.

--- Processing Frame 373 ---
DEBUG: YOLO found 1 total objects in this frame.
DEBUG: Detected class IDs are: [2]
DEBUG: After filtering for player class (ID 2), 

2025-07-02 21:11:48.396 | WARNING  | boxmot.motion.cmc.ecc:apply:99 - Affine matrix could not be generated: OpenCV(4.11.0) /io/opencv/modules/video/src/ecc.cpp:589: error: (-7:Iterations do not converge) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function 'findTransformECC'
. Returning identity
Processing Frames:  99%|█████████▉| 373/375 [02:55<00:00,  2.17it/s]2025-07-02 21:11:48.790 | WARNING  | boxmot.motion.cmc.ecc:apply:99 - Affine matrix could not be generated: OpenCV(4.11.0) /io/opencv/modules/video/src/ecc.cpp:589: error: (-7:Iterations do not converge) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function 'findTransformECC'
. Returning identity
2025-07-02 21:11:48.876 | WARNING  | boxmot.motion.cmc.ecc:apply:99 - Affine matrix could not be generated: OpenCV(4.11.0) /io/opencv/modules/video/src/ec

DEBUG: StrongSORT tracker returned 0 tracks for this frame.

--- Processing Frame 374 ---
DEBUG: YOLO found 1 total objects in this frame.
DEBUG: Detected class IDs are: [2]
DEBUG: After filtering for player class (ID 2), we have 1 detections.
DEBUG: StrongSORT tracker returned 0 tracks for this frame.

--- Processing Frame 375 ---
DEBUG: YOLO found 1 total objects in this frame.
DEBUG: Detected class IDs are: [2]
DEBUG: After filtering for player class (ID 2), we have 1 detections.
DEBUG: StrongSORT tracker returned 0 tracks for this frame.

Processing complete. Output video saved to: /content/drive/My Drive/detect/output_8.mp4


In [1]:

def get_even_numbers(lst):
    return [x for x in lst if x % 2]

print(get_even_numbers([1, 2, 3, 4, 5, 6]))

[1, 3, 5]


In [9]:
def flatten_and_sort(data):
    """
    Takes a list of lists and returns a single flattened, sorted list.
    """
    flattened = []
    for item in data:
        flattened.extend(item)
    return sorted(flattened)

In [10]:
data = [[1,2,5,4],[1,3,8,6,0]]

In [11]:
print(flatten_and_sort(data))

[0, 1, 1, 2, 3, 4, 5, 6, 8]
